## After noticing in the EDA that some of the images contain more than one type of clothing, I decided to analyze the product title with respect to the class label.

In [264]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [265]:
df = pd.read_json('/Users/butler/Documents/Image-Classification-Indian-Clothing/data/raw/test_data.json', lines=True)

In [266]:
df.head()

,image_url,image_path,brand,product_title,class_label,color
0,https://m.media-amazon.com/images/I/71NZQ-aaAS...,images/test/0.jpeg,TANTUJA BENGAL HANDLOOM,(A Govt. of West Bengal Enterprises) Handloom ...,saree,NaN
1,https://m.media-amazon.com/images/I/81aQmbWsG7...,images/test/1.jpeg,Trilok Fab,Women's Silk Saree With Blouse Piece,saree,NaN
2,https://m.media-amazon.com/images/I/71YjGvzs9y...,images/test/2.jpeg,Nine Sister,Women's Georgette Net Embroidered Saree with B...,saree,NaN
3,https://m.media-amazon.com/images/I/81JYhYV5Xr...,images/test/3.jpeg,Gauri Laxmi Enterprise,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,saree,NaN
4,https://m.media-amazon.com/images/I/91fXc5i5Dy...,images/test/4.jpeg,SOURBH,Women's Cotton Blend Digital Butta Printed Sar...,saree,NaN


In [267]:
df.tail()

,image_url,image_path,brand,product_title,class_label,color
7495,https://rukminim1.flixcart.com/image/309/371/k...,images/test/7495.jpeg,aaliya fashion,Women Embroidered Rayon Anarkali Kurta,women_kurta,Purple
7496,https://m.media-amazon.com/images/I/71xJFbz+uV...,images/test/7496.jpeg,AnjuShree Choice,Women's Rayon & Viscose Regular Stitched Kurta,women_kurta,NaN
7497,https://m.media-amazon.com/images/I/71CawDGew+...,images/test/7497.jpeg,W for Woman,Women's cotton straight Kurta,women_kurta,NaN
7498,https://m.media-amazon.com/images/I/61VBgoJQAt...,images/test/7498.jpeg,Vaamsi,Crepe Printed Kurti (VPK1805_Purple_Free Size),women_kurta,NaN
7499,https://m.media-amazon.com/images/I/612gggywBm...,images/test/7499.jpeg,Vbuyz,Women's Rayon Regular Kurta,women_kurta,NaN


In [268]:
df.shape

(7500, 6)

In [269]:
df.class_label.unique()

array(['saree', 'blouse', 'dhoti_pants', 'dupattas', 'gowns', 'kurta_men',
       'leggings_and_salwars', 'lehenga', 'mojaris_men', 'mojaris_women',
       'nehru_jackets', 'palazzos', 'petticoats', 'sherwanis',
       'women_kurta'], dtype=object)

In [270]:
# Making a copy of the df
df_revised = df.copy()

In [271]:
df_revised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   image_url      7500 non-null   object
 1   image_path     7500 non-null   object
 2   brand          7500 non-null   object
 3   product_title  7500 non-null   object
 4   class_label    7500 non-null   object
 5   color          905 non-null    object
dtypes: object(6)
memory usage: 351.7+ KB


In [272]:
# Reset index in case the number of the image is needed later
df_revised = df_revised.reset_index(names='number')

In [273]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color
0,0,https://m.media-amazon.com/images/I/71NZQ-aaAS...,images/test/0.jpeg,TANTUJA BENGAL HANDLOOM,(A Govt. of West Bengal Enterprises) Handloom ...,saree,NaN
1,1,https://m.media-amazon.com/images/I/81aQmbWsG7...,images/test/1.jpeg,Trilok Fab,Women's Silk Saree With Blouse Piece,saree,NaN
2,2,https://m.media-amazon.com/images/I/71YjGvzs9y...,images/test/2.jpeg,Nine Sister,Women's Georgette Net Embroidered Saree with B...,saree,NaN
3,3,https://m.media-amazon.com/images/I/81JYhYV5Xr...,images/test/3.jpeg,Gauri Laxmi Enterprise,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,saree,NaN
4,4,https://m.media-amazon.com/images/I/91fXc5i5Dy...,images/test/4.jpeg,SOURBH,Women's Cotton Blend Digital Butta Printed Sar...,saree,NaN


In [274]:
df_revised.tail()

,number,image_url,image_path,brand,product_title,class_label,color
7495,7495,https://rukminim1.flixcart.com/image/309/371/k...,images/test/7495.jpeg,aaliya fashion,Women Embroidered Rayon Anarkali Kurta,women_kurta,Purple
7496,7496,https://m.media-amazon.com/images/I/71xJFbz+uV...,images/test/7496.jpeg,AnjuShree Choice,Women's Rayon & Viscose Regular Stitched Kurta,women_kurta,NaN
7497,7497,https://m.media-amazon.com/images/I/71CawDGew+...,images/test/7497.jpeg,W for Woman,Women's cotton straight Kurta,women_kurta,NaN
7498,7498,https://m.media-amazon.com/images/I/61VBgoJQAt...,images/test/7498.jpeg,Vaamsi,Crepe Printed Kurti (VPK1805_Purple_Free Size),women_kurta,NaN
7499,7499,https://m.media-amazon.com/images/I/612gggywBm...,images/test/7499.jpeg,Vbuyz,Women's Rayon Regular Kurta,women_kurta,NaN


In [275]:
# Using column names with smallest part of clothing name
columns = ['kurta', 'women', 'men', 'dhoti', 'saree', 'sherwani', 'palazzo', 
           'mojari', 'nehru', 'legging', 'salwar', 'petticoat', 'blouse', 
           'dupatta', 'gown','lehenga']


In [276]:
# Adding these columns to the df with the values being true if the product title contains the column name
for column in columns:
    df_revised[str(column)] = df_revised.product_title.str.contains(str(column), flags=re.IGNORECASE)

In [277]:
# Checking for rows that have girl in the product title
df_revised['girl'] = df_revised.product_title.str.contains('girl', flags=re.IGNORECASE)

In [278]:
# Putting the rows that have girl in the product title into the women category
c1 = (df_revised.women == False)
c2 = (df_revised.girl == True)

df_revised.loc[c2 & c2, 'women'] = True

In [279]:
# Checking for rows that have woman in the product title
df_revised['woman'] = df_revised.product_title.str.contains('woman', flags=re.IGNORECASE)

In [280]:
# Putting the rows that have woman in the product title into the women category
c1 = (df_revised.women == False)
c2 = (df_revised.woman == True)

df_revised.loc[c2 & c2, 'women'] = True

In [281]:
# Checking for rows that have man in the product title
df_revised['man'] = df_revised.product_title.str.contains('man', flags=re.IGNORECASE)

In [282]:
# Checking for rows that have boy in the product title
df_revised['boy'] = df_revised.product_title.str.contains('boy', flags=re.IGNORECASE)

In [283]:
# Putting the rows that have man in the product title into the men category
c1 = (df_revised.men == False)
c2 = (df_revised.man == True)

df_revised.loc[c2 & c2, 'men'] = True

In [284]:
# Putting the rows that have boy in the product title into the men category
c1 = (df_revised.men == False)
c2 = (df_revised.boy == True)

df_revised.loc[c2 & c2, 'men'] = True

In [285]:
# No longer needed
df_revised.drop(['girl', 'woman', 'man', 'boy'], axis=1, inplace=True)
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,women,men,...,palazzo,mojari,nehru,legging,salwar,petticoat,blouse,dupatta,gown,lehenga
0,0,https://m.media-amazon.com/images/I/71NZQ-aaAS...,images/test/0.jpeg,TANTUJA BENGAL HANDLOOM,(A Govt. of West Bengal Enterprises) Handloom ...,saree,NaN,False,True,True,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/81aQmbWsG7...,images/test/1.jpeg,Trilok Fab,Women's Silk Saree With Blouse Piece,saree,NaN,False,True,True,...,False,False,False,False,False,False,True,False,False,False
2,2,https://m.media-amazon.com/images/I/71YjGvzs9y...,images/test/2.jpeg,Nine Sister,Women's Georgette Net Embroidered Saree with B...,saree,NaN,False,True,True,...,False,False,False,False,False,False,True,False,False,False
3,3,https://m.media-amazon.com/images/I/81JYhYV5Xr...,images/test/3.jpeg,Gauri Laxmi Enterprise,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,saree,NaN,False,True,True,...,False,False,False,False,False,False,True,False,False,False
4,4,https://m.media-amazon.com/images/I/91fXc5i5Dy...,images/test/4.jpeg,SOURBH,Women's Cotton Blend Digital Butta Printed Sar...,saree,NaN,False,True,True,...,False,False,False,False,False,False,True,False,False,False


In [286]:
# If the columns entitled 'men' and 'women' are both true, changing the 'men' column to false
c1 = (df_revised['men'] == True) 
c2 = (df_revised['women'] == True)

df_revised.loc[c1 & c2, 'men'] = False

In [287]:
# Getting the data for which the 'men' column is true now
df_revised_men = df_revised[df_revised.men == True]

In [288]:
# Making a column for women's kurtas
df_revised['women_kurta'] = [False]* 7500

In [289]:
# Making a column for men's kurtas
df_revised['kurta_men'] = [False]* 7500

In [290]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,women,men,...,nehru,legging,salwar,petticoat,blouse,dupatta,gown,lehenga,women_kurta,kurta_men
0,0,https://m.media-amazon.com/images/I/71NZQ-aaAS...,images/test/0.jpeg,TANTUJA BENGAL HANDLOOM,(A Govt. of West Bengal Enterprises) Handloom ...,saree,NaN,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/81aQmbWsG7...,images/test/1.jpeg,Trilok Fab,Women's Silk Saree With Blouse Piece,saree,NaN,False,True,False,...,False,False,False,False,True,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/71YjGvzs9y...,images/test/2.jpeg,Nine Sister,Women's Georgette Net Embroidered Saree with B...,saree,NaN,False,True,False,...,False,False,False,False,True,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/81JYhYV5Xr...,images/test/3.jpeg,Gauri Laxmi Enterprise,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,saree,NaN,False,True,False,...,False,False,False,False,True,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/91fXc5i5Dy...,images/test/4.jpeg,SOURBH,Women's Cotton Blend Digital Butta Printed Sar...,saree,NaN,False,True,False,...,False,False,False,False,True,False,False,False,False,False


In [291]:
# Making women_kurta true and kurta false if kurta is true and women is true
c1 = (df_revised['kurta'] == True) 
c2 = (df_revised['women'] == True)

df_revised.loc[c1 & c2, 'women_kurta'] = True
df_revised.loc[c1 & c2, 'kurta'] = False

In [292]:
# Making kurta_men true and kurta false if kurta is true and men is true
c1 = (df_revised['kurta'] == True) 
c2 = (df_revised['men'] == True)

df_revised.loc[c1 & c2, 'kurta_men'] = True
df_revised.loc[c1 & c2, 'kurta'] = False

In [293]:
# Since only women wear dupattas, if the class_label is dupatta and kurta is true, is must
# be a women's kurta
c1 = (df_revised.class_label == 'dupattas')
c2 = (df_revised.kurta == True)

df_revised.loc[c1 & c2, 'women_kurta'] = True
df_revised.loc[c1 & c2, 'kurta'] = False

In [294]:
# Making kurta false when if kurta is true and kurta_men is the class label
c1 = (df_revised['kurta'] == True) 
c2 = (df_revised.class_label == 'kurta_men')

df_revised.loc[c1 & c2, 'kurta'] = False

In [295]:
# Making kurta false when if kurta is true and women_kurta is the class label
c1 = (df_revised['kurta'] == True) 
c2 = (df_revised.class_label == 'women_kurta')

df_revised.loc[c1 & c2, 'kurta'] = False

In [296]:
df_revised.kurta.value_counts()

kurta
False    7484
True       16
Name: count, dtype: int64

In [297]:
(df_revised.kurta == 'kurta').value_counts()

kurta
False    7500
Name: count, dtype: int64

## This leaves 16 rows where kurta is true, men and women are false, and the class label does not contain kurta

In [298]:
# Making columns for mojaris_men, mojaris_women, and leggings_and_salwars
df_revised['mojaris_men'] = [False]*7500
df_revised['mojaris_women'] = [False]*7500
df_revised['leggings_and_salwars'] = [False]*7500

In [299]:
# mojaris_women column is true if mojaris and women are true
c1 = (df_revised['mojari'] == True) 
c2 = (df_revised['women'] == True)

df_revised.loc[c1 & c2, 'mojaris_women'] = True

In [300]:
# mojaris_men column is true if mojaris and men are true
c1 = (df_revised['mojari'] == True) 
c2 = (df_revised['men'] == True)

df_revised.loc[c1 & c2, 'mojaris_men'] = True

In [301]:
# Putting the columns legging and salwar together
c1 = (df_revised['legging'] == True) 
c2 = (df_revised['salwar'] == True)

df_revised.loc[c1 | c2, 'leggings_and_salwars'] = True

In [302]:
# Dropping the columns that are no longer needed
df_revised.drop(['men', 'women', 'legging','salwar','mojari'], axis=1, inplace=True)

In [303]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti,saree,...,petticoat,blouse,dupatta,gown,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
0,0,https://m.media-amazon.com/images/I/71NZQ-aaAS...,images/test/0.jpeg,TANTUJA BENGAL HANDLOOM,(A Govt. of West Bengal Enterprises) Handloom ...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/81aQmbWsG7...,images/test/1.jpeg,Trilok Fab,Women's Silk Saree With Blouse Piece,saree,NaN,False,False,True,...,False,True,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/71YjGvzs9y...,images/test/2.jpeg,Nine Sister,Women's Georgette Net Embroidered Saree with B...,saree,NaN,False,False,True,...,False,True,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/81JYhYV5Xr...,images/test/3.jpeg,Gauri Laxmi Enterprise,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,saree,NaN,False,False,True,...,False,True,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/91fXc5i5Dy...,images/test/4.jpeg,SOURBH,Women's Cotton Blend Digital Butta Printed Sar...,saree,NaN,False,False,True,...,False,True,False,False,False,False,False,False,False,False


In [304]:
# Removing the blouse label from all sarees in class_label because 
# models wearing sarees cannot be shown without a saree blouse

c1 = (df_revised['class_label'] == 'saree') 
c2 = (df_revised['blouse'] == True)

df_revised.loc[c1 & c2, 'blouse'] = False

In [305]:
# Removing the saree label from all blouses in class_label because 
# blouses are blouses for sarees or lehengas. Therefore, it is likely that saree 
# will be in the product description

c1 = (df_revised['class_label'] == 'blouse') 
c2 = (df_revised['saree'] == True)

df_revised.loc[c1 & c2, 'saree'] = False

In [306]:
# Removing the saree label from all blouses in class_label because 
# blouses are blouses for sarees or lehengas. Therefore, it is likely that lehenga 
# will be in the product description

c1 = (df_revised['class_label'] == 'blouse') 
c2 = (df_revised['lehenga'] == True)

df_revised.loc[c1 & c2, 'lehenga'] = False

In [307]:
# Gowns are sometimes called dresses
df_revised['dress'] = df_revised.product_title.str.contains('dress', flags=re.IGNORECASE)

In [308]:
# If dress is true and gowns is not, put the value for gowns as True in gown column
c1 = (df_revised.gown == False)
c2 = (df_revised.dress == True)

df_revised.loc[c1 & c2, 'gown'] = True

In [309]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti,saree,...,blouse,dupatta,gown,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars,dress
0,0,https://m.media-amazon.com/images/I/71NZQ-aaAS...,images/test/0.jpeg,TANTUJA BENGAL HANDLOOM,(A Govt. of West Bengal Enterprises) Handloom ...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/81aQmbWsG7...,images/test/1.jpeg,Trilok Fab,Women's Silk Saree With Blouse Piece,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/71YjGvzs9y...,images/test/2.jpeg,Nine Sister,Women's Georgette Net Embroidered Saree with B...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/81JYhYV5Xr...,images/test/3.jpeg,Gauri Laxmi Enterprise,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/91fXc5i5Dy...,images/test/4.jpeg,SOURBH,Women's Cotton Blend Digital Butta Printed Sar...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [310]:
df_revised.loc[1]

number                                                                  1
image_url               https://m.media-amazon.com/images/I/81aQmbWsG7...
image_path                                             images/test/1.jpeg
brand                                                          Trilok Fab
product_title                        Women's Silk Saree With Blouse Piece
class_label                                                         saree
color                                                                 NaN
kurta                                                               False
dhoti                                                               False
saree                                                                True
sherwani                                                            False
palazzo                                                             False
nehru                                                               False
petticoat                             

In [311]:
# Renaming columns to conform to class labels
df_revised = df_revised.rename({'dhoti':'dhoti_pants', 'sherwani':'sherwanis', 'palazzo':'palazzos', 
             'nehru':'nehru_jackets', 'petticoat':'petticoats', 'dupatta':'dupattas', 
             'gown':'gowns'},axis=1)
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars,dress
0,0,https://m.media-amazon.com/images/I/71NZQ-aaAS...,images/test/0.jpeg,TANTUJA BENGAL HANDLOOM,(A Govt. of West Bengal Enterprises) Handloom ...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/81aQmbWsG7...,images/test/1.jpeg,Trilok Fab,Women's Silk Saree With Blouse Piece,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/71YjGvzs9y...,images/test/2.jpeg,Nine Sister,Women's Georgette Net Embroidered Saree with B...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/81JYhYV5Xr...,images/test/3.jpeg,Gauri Laxmi Enterprise,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/91fXc5i5Dy...,images/test/4.jpeg,SOURBH,Women's Cotton Blend Digital Butta Printed Sar...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [312]:
# Changing each new column to false if the class_label is the same as the column name
columns = ['dhoti_pants', 'saree', 'sherwanis', 'palazzos', 'nehru_jackets', 'petticoats',
           'blouse','dupattas', 'gowns', 'lehenga', 'women_kurta', 'kurta_men', 
           'mojaris_men', 'mojaris_women', 'leggings_and_salwars']

for column in columns:
    c1 = (df_revised[str(column)] == True)
    c2 = (df_revised['class_label'] == str(column))
    df_revised.loc[c1 & c2, str(column)] = False

df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars,dress
0,0,https://m.media-amazon.com/images/I/71NZQ-aaAS...,images/test/0.jpeg,TANTUJA BENGAL HANDLOOM,(A Govt. of West Bengal Enterprises) Handloom ...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/81aQmbWsG7...,images/test/1.jpeg,Trilok Fab,Women's Silk Saree With Blouse Piece,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/71YjGvzs9y...,images/test/2.jpeg,Nine Sister,Women's Georgette Net Embroidered Saree with B...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/81JYhYV5Xr...,images/test/3.jpeg,Gauri Laxmi Enterprise,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/91fXc5i5Dy...,images/test/4.jpeg,SOURBH,Women's Cotton Blend Digital Butta Printed Sar...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [313]:
columns = ['dhoti_pants', 'saree', 'kurta', 'sherwanis', 'palazzos', 'nehru_jackets', 'petticoats',
           'blouse','dupattas', 'gowns', 'lehenga', 'women_kurta', 'kurta_men', 
           'mojaris_men', 'mojaris_women', 'leggings_and_salwars']
# If a column is true, putting the column name as the value; otherwise, 0
def func(x):
    if x == True:
        return str(column)
    else:
        return 0

for column in columns:
    df_revised[str(column)] = df_revised[str(column)].apply(lambda x: func(x))

In [314]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars,dress
0,0,https://m.media-amazon.com/images/I/71NZQ-aaAS...,images/test/0.jpeg,TANTUJA BENGAL HANDLOOM,(A Govt. of West Bengal Enterprises) Handloom ...,saree,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,False
1,1,https://m.media-amazon.com/images/I/81aQmbWsG7...,images/test/1.jpeg,Trilok Fab,Women's Silk Saree With Blouse Piece,saree,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,False
2,2,https://m.media-amazon.com/images/I/71YjGvzs9y...,images/test/2.jpeg,Nine Sister,Women's Georgette Net Embroidered Saree with B...,saree,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,False
3,3,https://m.media-amazon.com/images/I/81JYhYV5Xr...,images/test/3.jpeg,Gauri Laxmi Enterprise,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,saree,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,False
4,4,https://m.media-amazon.com/images/I/91fXc5i5Dy...,images/test/4.jpeg,SOURBH,Women's Cotton Blend Digital Butta Printed Sar...,saree,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,False


In [315]:
df_revised.tail()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars,dress
7495,7495,https://rukminim1.flixcart.com/image/309/371/k...,images/test/7495.jpeg,aaliya fashion,Women Embroidered Rayon Anarkali Kurta,women_kurta,Purple,0,0,0,...,0,0,0,0,0,0,0,0,0,False
7496,7496,https://m.media-amazon.com/images/I/71xJFbz+uV...,images/test/7496.jpeg,AnjuShree Choice,Women's Rayon & Viscose Regular Stitched Kurta,women_kurta,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,False
7497,7497,https://m.media-amazon.com/images/I/71CawDGew+...,images/test/7497.jpeg,W for Woman,Women's cotton straight Kurta,women_kurta,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,False
7498,7498,https://m.media-amazon.com/images/I/61VBgoJQAt...,images/test/7498.jpeg,Vaamsi,Crepe Printed Kurti (VPK1805_Purple_Free Size),women_kurta,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,False
7499,7499,https://m.media-amazon.com/images/I/612gggywBm...,images/test/7499.jpeg,Vbuyz,Women's Rayon Regular Kurta,women_kurta,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,False


In [316]:
# Removing columns not needed
df_revised = df_revised.drop(['number', 'image_url', 'brand', 'color', 'dress'], axis=1)

In [317]:
# Saving revised dataframe
df_revised.to_csv('/Users/butler/Documents/Image-Classification-Indian-Clothing/data/interim/test_other_categories.csv')

### Making a dataframe for each label that is different than the class label and exploring the product titles and class labels of some of the rows

### Sarees

In [318]:
df_saree = df_revised[df_revised.saree == 'saree']
df_saree.shape

(191, 19)

In [319]:
df_saree.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
2056,images/test/2056.jpeg,Saree for women | Women's Fancy Saree | Silk S...,gowns,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
2090,images/test/2090.jpeg,Chiffon with Blouse Piece Saree (Riva516_Yello...,gowns,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
2265,images/test/2265.jpeg,DaSync Silver Stone Hanging latkan Laces Borde...,gowns,0,0,saree,0,0,0,0,blouse,0,0,lehenga,0,0,0,0,0
2397,images/test/2397.jpeg,Women's Saree with blouse piece,gowns,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
2412,images/test/2412.jpeg,Women's Satin Saree With Blouse Piece,gowns,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0


### Mislabeled:

In [320]:
print(f'Product title: {df_saree.iloc[0,1]}')
print(f'Class label: {df_saree.iloc[0,2]}')

Product title: Saree for women | Women's Fancy Saree | Silk Saree With Silk Blouse Women's Saree (PARENT-ZQ-152)
Class label: gowns


In [321]:
print(f'Product title: {df_saree.iloc[3,1]}')
print(f'Class label: {df_saree.iloc[3,2]}')

Product title: Women's Saree with blouse piece
Class label: gowns


In [322]:
print(f'Product title: {df_saree.iloc[4,1]}')
print(f'Class label: {df_saree.iloc[4,2]}')

Product title: Women's Satin Saree With Blouse Piece
Class label: gowns


### Blouses

In [323]:
df_blouse = df_revised[df_revised.blouse == 'blouse']
df_blouse.shape

(201, 19)

In [324]:
df_blouse.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1769,images/test/1769.jpeg,Women'S Kashmiri Art Printed Shawl Dupatta Wit...,dupattas,0,0,0,0,0,0,0,blouse,0,0,0,0,0,0,0,0
2056,images/test/2056.jpeg,Saree for women | Women's Fancy Saree | Silk S...,gowns,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
2069,images/test/2069.jpeg,Chlamys Women Floral Printed Crape Unstiched F...,gowns,0,0,0,0,0,0,0,blouse,0,0,0,0,0,0,0,0
2090,images/test/2090.jpeg,Chiffon with Blouse Piece Saree (Riva516_Yello...,gowns,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
2115,images/test/2115.jpeg,Woman's/Girl's Ethnic Unstitched Art Crepe Pri...,gowns,0,0,0,0,palazzos,0,0,blouse,0,0,0,0,0,0,0,0


### Mislabeled:

In [325]:
print(f'Product title: {df_blouse.iloc[3,1]}')
print(f'Class label: {df_blouse.iloc[3,2]}')

Product title: Chiffon with Blouse Piece Saree (Riva516_Yellow_Free Size)
Class label: gowns


### Dhoti Pants

In [326]:
df_dhoti_pants = df_revised[df_revised.dhoti_pants == 'dhoti_pants']
df_dhoti_pants.shape

(45, 19)

In [327]:
df_dhoti_pants.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
117,images/test/117.jpeg,Women's Olive Green Poly Crepe Crop Top And Dh...,saree,0,dhoti_pants,0,0,0,0,0,0,dupattas,0,0,0,0,0,0,0
2328,images/test/2328.jpeg,Women's Regular Fit Dhoti Pants,gowns,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2541,images/test/2541.jpeg,Men Beige Dhoti Pants,kurta_men,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2735,images/test/2735.jpeg,Men Beige Dupion Silk Dhoti & Maroon Kurta,kurta_men,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2782,images/test/2782.jpeg,Men Green Solid Kurta with Dhoti Pants,kurta_men,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Mislabeled:

In [328]:
print(f'Product title: {df_dhoti_pants.iloc[1,1]}')
print(f'Class label: {df_dhoti_pants.iloc[1,2]}')

Product title: Women's Regular Fit Dhoti Pants
Class label: gowns


In [329]:
print(f'Product title: {df_dhoti_pants.iloc[2,1]}')
print(f'Class label: {df_dhoti_pants.iloc[2,2]}')

Product title: Men Beige Dhoti Pants
Class label: kurta_men


### Dupattas

In [330]:
df_dupattas = df_revised[df_revised.dupattas == 'dupattas']
df_dupattas.shape

(278, 19)

In [331]:
df_dupattas.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
117,images/test/117.jpeg,Women's Olive Green Poly Crepe Crop Top And Dh...,saree,0,dhoti_pants,0,0,0,0,0,0,dupattas,0,0,0,0,0,0,0
262,images/test/262.jpeg,Women's Jacquard Banarasi Silk Dupatta,saree,0,0,0,0,0,0,0,0,dupattas,0,0,0,0,0,0,0
1002,images/test/1002.jpeg,Women Cotton Floral and Embroidered Pattern St...,dhoti_pants,0,0,0,0,0,0,0,0,dupattas,0,0,women_kurta,0,0,0,0
1017,images/test/1017.jpeg,Women’s Pure Cotton Plain Semi Patiala Dhoti S...,dhoti_pants,0,0,0,0,0,0,0,0,dupattas,0,0,0,0,0,0,leggings_and_salwars
1032,images/test/1032.jpeg,Women’s Pure Cotton Plain Semi Patiala Dhoti S...,dhoti_pants,0,0,0,0,0,0,0,0,dupattas,0,0,0,0,0,0,leggings_and_salwars


### Mislabeled:

In [332]:
print(f'Product title: {df_dupattas.iloc[0,1]}')
print(f'Class label: {df_dupattas.iloc[0,2]}')

Product title: Women's Olive Green Poly Crepe Crop Top And Dhoti With Attached Dupatta
Class label: saree


In [333]:
print(f'Product title: {df_dupattas.iloc[1,1]}')
print(f'Class label: {df_dupattas.iloc[1,2]}')

Product title: Women's Jacquard Banarasi Silk Dupatta
Class label: saree


### Gowns

In [334]:
df_gowns = df_revised[df_revised.gowns == 'gowns']
df_gowns.shape

(63, 19)

In [335]:
df_gowns.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
568,images/test/568.jpeg,Women/Girls Ethnic Unstitched Chanderi Cotton ...,blouse,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
777,images/test/777.jpeg,Women/Girls Ethnic Unstitched Chanderi Cotton ...,blouse,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
1211,images/test/1211.jpeg,"Women's Satin 2 Piece Night Suit Dress (Black,...",dhoti_pants,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
1341,images/test/1341.jpeg,Girls Western Dresses Cotton Blend Top and Den...,dhoti_pants,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
1533,images/test/1533.jpeg,Woman's Jacquard Lining Designed Net Dupatta w...,dupattas,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0


In [336]:
print(f'Product title: {df_gowns.iloc[2,1]}')
print(f'Class label: {df_gowns.iloc[2,2]}')

Product title: Women's Satin 2 Piece Night Suit Dress (Black, Free Size)
Class label: dhoti_pants


### Kurtas

In [337]:
df_kurta = df_revised[df_revised.kurta == 'kurta']
df_kurta.shape

(16, 19)

In [338]:
df_kurta.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1181,images/test/1181.jpeg,Dupion Silk Full Sleeve Long Kurta Blended Dho...,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1335,images/test/1335.jpeg,Dupion Silk Full Sleeve Long Kurta Blended Dho...,dhoti_pants,kurta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1379,images/test/1379.jpeg,Black Chanderi Silk Kurta with organza hand wo...,dhoti_pants,kurta,0,0,0,0,0,0,0,dupattas,0,0,0,0,0,0,0
2107,images/test/2107.jpeg,Impex Ancle Length Kurta,gowns,kurta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2239,images/test/2239.jpeg,Faux Georgette Embroidered Kurta & Palazzo Mat...,gowns,kurta,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0


### Mislabeled:

In [339]:
print(f'Product title: {df_kurta.iloc[3,1]}')
print(f'Class label: {df_kurta.iloc[3,2]}')

Product title: Impex Ancle Length Kurta
Class label: gowns


### Men's Kurtas

In [340]:
df_kurta_men = df_revised[df_revised.kurta_men == 'kurta_men']
df_kurta_men.shape

(61, 19)

In [341]:
df_kurta_men.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1120,images/test/1120.jpeg,Men's Ethnic Rayon Side Button Long Straight K...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0
1222,images/test/1222.jpeg,Baap Beta Maroon Kurta and Dhoti Pant Set (Man...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0
1235,images/test/1235.jpeg,Mens Silk Self Design Kurta And Dhoti Pant (Blue),dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0
1252,images/test/1252.jpeg,Men's Regular Kurta,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0
1269,images/test/1269.jpeg,Mens Self Design Kurta and Dhoti Pant Set,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0


### Mislabeled:

In [342]:
print(f'Product title: {df_kurta_men.iloc[0,1]}')
print(f'Class label: {df_kurta_men.iloc[0,2]}')

Product title: Men's Ethnic Rayon Side Button Long Straight Kurta
Class label: dhoti_pants


In [343]:
print(f'Product title: {df_kurta_men.iloc[3,1]}')
print(f'Class label: {df_kurta_men.iloc[3,2]}')

Product title: Men's Regular Kurta
Class label: dhoti_pants


### Leggings and Salwars

In [344]:
df_leggings_and_salwars = df_revised[df_revised.leggings_and_salwars == 'leggings_and_salwars']
df_leggings_and_salwars.shape

(214, 19)

In [345]:
df_leggings_and_salwars.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
234,images/test/234.jpeg,The Boutique House - Soft malmal cotton sarees...,saree,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
1003,images/test/1003.jpeg,Women's Lycra Side Plated Dhoti Patiala Salwar...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
1004,images/test/1004.jpeg,Women's Cotton Viscose Lycra Dhoti Patiyala Sa...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
1007,images/test/1007.jpeg,Women's Viscose Lycra Cotton Dhoti Patiyala Sa...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
1017,images/test/1017.jpeg,Women’s Pure Cotton Plain Semi Patiala Dhoti S...,dhoti_pants,0,0,0,0,0,0,0,0,dupattas,0,0,0,0,0,0,leggings_and_salwars


In [346]:
print(f'Product title: {df_leggings_and_salwars.iloc[0,1]}')
print(f'Class label: {df_leggings_and_salwars.iloc[0,2]}')

Product title: The Boutique House - Soft malmal cotton sarees with zari checks with embroidery Work - Kota Doria cotton saree - Kota Doria Lehanga - Salwar suits - Ethnic Wear - Pattern 3
Class label: saree


### Lehenga

In [347]:
df_lehenga = df_revised[df_revised.lehenga == 'lehenga']
df_lehenga.shape

(29, 19)

In [348]:
df_lehenga.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1522,images/test/1522.jpeg,Women and Girls Latest Design Chanderi Dupatta...,dupattas,0,0,0,0,0,0,0,0,0,0,lehenga,women_kurta,0,0,0,0
1527,images/test/1527.jpeg,"Girl's Embroidered Long Jacket, Lehenga and Du...",dupattas,0,0,0,0,0,0,0,0,0,0,lehenga,0,0,0,0,0
1535,images/test/1535.jpeg,Women's Silk semi-stitched Lehenga Choli with ...,dupattas,0,0,0,0,0,0,0,0,0,0,lehenga,0,0,0,0,0
1646,images/test/1646.jpeg,Celebrity Women's Net Embroidered Lehenga Chol...,dupattas,0,0,0,0,0,0,0,0,0,0,lehenga,0,0,0,0,0
1665,images/test/1665.jpeg,Digital Printed Chandei Latest Floral Design D...,dupattas,0,0,0,0,0,0,0,0,0,0,lehenga,women_kurta,0,0,0,0


In [349]:
print(f'Product title: {df_lehenga.iloc[2,1]}')
print(f'Class label: {df_lehenga.iloc[2,2]}')

Product title: Women's Silk semi-stitched Lehenga Choli with Dupatta (Guldasta1K, Pink, Free Size)
Class label: dupattas


### Men's Mojaris

In [350]:
df_mojaris_men = df_revised[df_revised.mojaris_men == 'mojaris_men']
df_mojaris_men.shape

(16, 19)

In [351]:
df_mojaris_men.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
4559,images/test/4559.jpeg,Men Casual Shoe Style Ethnic and Traditional B...,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
4564,images/test/4564.jpeg,Stylish Casual Ethnic Nagra/Mojaris/Jutti Form...,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
4588,images/test/4588.jpeg,Men's Ethnic Mojari/Juttis/Nagra/Loafers,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
4606,images/test/4606.jpeg,Unique Craft Men's Leather Mojaris,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
4608,images/test/4608.jpeg,Wedding Mojaris Shoes for Men Loafer Stylish B...,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0


### Many of the images labeled mojaris_women are actually men's

In [352]:
print(f'Product title: {df_mojaris_men.iloc[0,1]}')
print(f'Class label: {df_mojaris_men.iloc[0,2]}')

Product title: Men Casual Shoe Style Ethnic and Traditional Black Leather Jutti Mojari
Class label: mojaris_women


In [353]:
print(f'Product title: {df_mojaris_men.iloc[1,1]}')
print(f'Class label: {df_mojaris_men.iloc[1,2]}')

Product title: Stylish Casual Ethnic Nagra/Mojaris/Jutti Formal Shoes for Men's
Class label: mojaris_women


### Women's Mojaris

In [354]:
df_mojaris_women = df_revised[df_revised.mojaris_women == 'mojaris_women']
df_mojaris_women.shape

(5, 19)

In [355]:
df_mojaris_women.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
4035,images/test/4035.jpeg,"Juttiwala Women's Ethnic Mojari, Juti, Bellies...",mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0
4112,images/test/4112.jpeg,Women's Fabric Mojaris,mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0
4302,images/test/4302.jpeg,Women Mojari Handicraft Slipper Shoes,mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0
4355,images/test/4355.jpeg,Juttiwala Women's Traditional Slip-On Mojari J...,mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0
4464,images/test/4464.jpeg,Ballet Flats Slip On Mojari Sandal Indian Jutt...,mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0


### Many of the images labeled mojaris_men are actually women's

In [356]:
print(f'Product title: {df_mojaris_women.iloc[0,1]}')
print(f'Class label: {df_mojaris_women.iloc[0,2]}')

Product title: Juttiwala Women's Ethnic Mojari, Juti, Bellies, Morpankh Design for Woman & Girl Bellies
Class label: mojaris_men


In [357]:
print(f'Product title: {df_mojaris_women.iloc[1,1]}')
print(f'Class label: {df_mojaris_women.iloc[1,2]}')

Product title: Women's Fabric Mojaris
Class label: mojaris_men


### Nehru Jackets

In [358]:
df_nehru_jackets = df_revised[df_revised.nehru_jackets == 'nehru_jackets']
df_nehru_jackets.shape

(17, 19)

In [359]:
df_nehru_jackets.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
645,images/test/645.jpeg,KULLU Jacket Nehru Jacket Reversal Both Side U...,blouse,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0
2502,images/test/2502.jpeg,Men Pink &amp; Maroon Solid Kurta with Churida...,kurta_men,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0
2525,images/test/2525.jpeg,Men Teal &amp; Off-White Solid Kurta with Chur...,kurta_men,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0
2534,images/test/2534.jpeg,Men Off-White &amp; Peach-Coloured Solid Kurta...,kurta_men,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0
2551,images/test/2551.jpeg,Printed Full Sleeves Kurta and Churidar Set wi...,kurta_men,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0


### Mislabeled:

In [360]:
print(f'Product title: {df_nehru_jackets.iloc[0,1]}')
print(f'Class label: {df_nehru_jackets.iloc[0,2]}')

Product title: KULLU Jacket Nehru Jacket Reversal Both Side USABLE for Men ONE Side Grey Other Side KINGRY Black
Class label: blouse


### Palazzos

In [361]:
df_palazzos = df_revised[df_revised.palazzos == 'palazzos']
df_palazzos.shape

(182, 19)

In [362]:
df_palazzos.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1018,images/test/1018.jpeg,"Women's Regular Fit Full Length, Design for Mu...",dhoti_pants,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0
1034,images/test/1034.jpeg,Women Rayon Flared Garara/Sharara Palazzo Pant...,dhoti_pants,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0
1041,images/test/1041.jpeg,"Women's Regular Fit Full Length, Design for Mu...",dhoti_pants,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0
1057,images/test/1057.jpeg,R & B Enterprises Stylish Solid Pink Palazzo f...,dhoti_pants,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0
1091,images/test/1091.jpeg,Women's Viscose Palazzo Bottom Pants,dhoti_pants,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0


In [363]:
print(f'Product title: {df_palazzos.iloc[2,1]}')
print(f'Class label: {df_palazzos.iloc[2,2]}')

Product title: Women's Regular Fit Full Length, Design for Multicolor Plain Solid Dhoti,Haram Pant Bottom Wear With Beautiful Flared Sarara/Garara/Palazzo/Free Size Combo (Pack Of 2)
Class label: dhoti_pants


### Petticoats

In [364]:
df_petticoats = df_revised[df_revised.petticoats == 'petticoats']
df_petticoats.shape

(1, 19)

In [365]:
df_petticoats.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
432,images/test/432.jpeg,Women's Rani Pink Lycra Shapewear Saree Petticoat,saree,0,0,0,0,0,0,petticoats,0,0,0,0,0,0,0,0,0


### Mislabeled:

In [366]:
print(f'Product title: {df_petticoats.iloc[0,1]}')
print(f'Class label: {df_petticoats.iloc[0,2]}')

Product title: Women's Rani Pink Lycra Shapewear Saree Petticoat
Class label: saree


### Sherwanis

In [367]:
df_sherwanis = df_revised[df_revised.sherwanis == 'sherwanis']
df_sherwanis.shape

(50, 19)

In [368]:
df_sherwanis.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
2686,images/test/2686.jpeg,Men's Silk Sherwani Straight Kurta,kurta_men,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0
2726,images/test/2726.jpeg,Men's Jacquard Silk Floral Print Indowestern S...,kurta_men,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0
4008,images/test/4008.jpeg,Black Velvet Wedding & Sherwani Mojari's,mojaris_men,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0
4018,images/test/4018.jpeg,Jutti for Men's Mens Mojari Indian Shoes Handm...,mojaris_men,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0
4048,images/test/4048.jpeg,Men Royal Luxury & Traditional Designing Handc...,mojaris_men,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0


### Mislabeled: It is a sherwani-style kurta, not a sherwani

In [369]:
print(f'Product title: {df_sherwanis.iloc[0,1]}')
print(f'Class label: {df_sherwanis.iloc[0,2]}')

Product title: Men's Silk Sherwani Straight Kurta
Class label: kurta_men


### Women's Kurtas

In [370]:
df_women_kurta = df_revised[df_revised.women_kurta == 'women_kurta']
df_women_kurta.shape

(169, 19)

In [371]:
df_women_kurta.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
111,images/test/111.jpeg,Women's georgette with blouse piece Kurta (Kur...,saree,0,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
1002,images/test/1002.jpeg,Women Cotton Floral and Embroidered Pattern St...,dhoti_pants,0,0,0,0,0,0,0,0,dupattas,0,0,women_kurta,0,0,0,0
1016,images/test/1016.jpeg,Women's Black Pure Cotton Straight Kurta With ...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
1027,images/test/1027.jpeg,Rayon Kurta and Multicolor Dhoti Pant Set For ...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
1028,images/test/1028.jpeg,Women's Sage Green Umbrella Print Rayon Kurta ...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0


In [372]:
print(f'Product title: {df_women_kurta.iloc[0,1]}')
print(f'Class label: {df_women_kurta.iloc[0,2]}')

Product title: Women's georgette with blouse piece Kurta (Kurtas321_ Rani_ 6 Meter)
Class label: saree


In [373]:
print(f'Product title: {df_women_kurta.iloc[1,1]}')
print(f'Class label: {df_women_kurta.iloc[1,2]}')

Product title: Women Cotton Floral and Embroidered Pattern Straight Kurta and Pant with Dupatta Set (STYLEP)
Class label: dhoti_pants


## While not all of the items in each dataframe are mislabeled, many are. 
## There are 191 rows in the saree dataframe;
## 201 rows in the blouse dataframe;
## 45 rows in the dhoti_pants dataframe;
## 278 rows in the dupatta dataframe;
## 174 rows in the gown dataframe;
## 16 rows in the kurta dataframe;
## 61 rows in the kurta_men dataframe;
## 214 rows in the leggings_and_salwars dataframe;
## 29 rows in the lehenga dataframe;
## 16 rows in the mojaris_men dataframe;
## 5 rows in the mojaris_women dataframe;
## 17 rows in the Nehru_jackets dataframe;
## 182 rows in the palazzos dataframe;
## 1 row in the petticoats dataframe;
## 50 rows in the sherwani dataframe;
## and 169 rows in the women_kurta dataframe.